In [34]:
#from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.metrics import Accuracy
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import model_selection
#https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

#drive.mount('/content/drive')

In [35]:
#DATA PREPARATION
raw_datas = pd.read_csv("/Users/asus/notebook/TA/Dann Dataset/PIR dataset Time train.csv")
number_of_day = raw_datas.shape[0]/86400
print("Total data = " + str(raw_datas.shape[0]) + " detik")
print("Total hari dalam data = " + str(int(number_of_day)) + " hari")

Total data = 604800 detik
Total hari dalam data = 7 hari


In [13]:
#EXTRACTING DATA

datas = pd.read_csv("/Users/asus/notebook/TA/Dann Dataset/PIR dataset Duration train.csv")
file = open("/Users/asus/notebook/TA/Dann Dataset/PIR last 30 list dataset train.csv","w")
for z in range(30):
  file.write("Durasi")
  file.write(str(z+1))
  for y in range(4):
    file.write(",Data")
    file.write(str(z+1))
    file.write("|")
    file.write("Pir")
    file.write(str(y+1))
  file.write(",")
file.write("Gerakan")
file.write("\n")

percent1 = round(datas.shape[0]/100)
percentage = 0

for i in range(datas.shape[0]):
  if(i%percent1 == 0):
    print(str(percentage)+"%")
    percentage+=1
  if(i < 29):
    for u in range(29-i):
      file.write("0,0,0,0,0,")
    for e in range(i+1):
      for t in range(5):
        file.write(str(datas.iloc[e][t]))
        file.write(",") 
  else:
    for a in range(30):
      for o in range(5):
        file.write(str(datas.iloc[i-(29-a)][o]))
        file.write(",")
  file.write(datas.iloc[i][5])
  file.write("\n")

print("100%")
file.close()

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
100%


In [36]:
# LOAD DATASET
feature_data =pd.read_csv("/Users/asus/notebook/TA/Dann Dataset/PIR last 30 list dataset train.csv")

# ADD LABEL ENCODER
label_encoder = LabelEncoder()
feature_data['Gerakan'] = label_encoder.fit_transform(feature_data['Gerakan'])
datas_labels = label_encoder.classes_

In [37]:
# LIST OF LABELS
print(datas_labels)

['LOITERING' 'NORMAL']


In [38]:
x = feature_data.drop('Gerakan',axis=1)
y = feature_data['Gerakan']

# RESHAPE DATASET
x = x.values.reshape(-1,30,5)   #INPUT 4
y = y.values.reshape((-1,1))

# SPLIT DATASET
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) #Split Dataset Train 70% dan Test 30%

# CHECK THE SHAPE
print("Train:", x_train.shape , y_train.shape)
print("Test:",x_test.shape , y_test.shape)

# SET TOTAL FOLD
total_fold = 5

Train: (140202, 30, 5) (140202, 1)
Test: (60087, 30, 5) (60087, 1)


In [21]:
#RNN BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4), #metode optimizer default
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10
1122/1122 - 32s - loss: 0.6744 - accuracy: 0.5713 - 32s/epoch - 28ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.6010 - accuracy: 0.6622 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.5612 - accuracy: 0.6961 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.5408 - accuracy: 0.7083 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.5265 - accuracy: 0.7151 - 33s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.5094 - accuracy: 0.7268 - 33s/epoch - 30ms/step
Epoch 7/10
1122/1122 - 34s - loss: 0.4785 - accuracy: 0.7534 - 34s/epoch - 30ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.4069 - accuracy: 0.8140 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.3330 - accuracy: 0.8627 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 35s - loss: 0.2740 - accuracy: 0.8940 - 35s/epoch - 31ms/step
accuracy: 81.52%
INFO:tensorflow:Assets written to: ram://09ab0c80-b43a-4bf5-8497-3768726b1dd0/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 37s - loss: 0.6736 - accuracy: 0.5752 - 37s/epoch - 33ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.5973 - accuracy: 0.6670 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.5457 - accuracy: 0.7128 - 33s/epoch - 30ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.5072 - accuracy: 0.7429 - 33s/epoch - 30ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.4576 - accuracy: 0.7830 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3640 - accuracy: 0.8500 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.2957 - accuracy: 0.8851 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2408 - accuracy: 0.9118 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 34s - loss: 0.2075 - accuracy: 0.9249 - 34s/epoch - 30ms/step
Epoch 10/10
1122/1122 - 36s - loss: 0.1845 - accuracy: 0.9337 - 36s/epoch - 32ms/step
accuracy: 93.98%
INFO:tensorflow:Assets written to: ram://3b4116f5-ec15-4963-bb2e-e5fc49f3654d/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 38s - loss: 0.6951 - accuracy: 0.5505 - 38s/epoch - 34ms/step
Epoch 2/10
1122/1122 - 40s - loss: 0.5631 - accuracy: 0.7167 - 40s/epoch - 36ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.4263 - accuracy: 0.8196 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 34s - loss: 0.3404 - accuracy: 0.8639 - 34s/epoch - 31ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.2738 - accuracy: 0.8963 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.2308 - accuracy: 0.9159 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.1975 - accuracy: 0.9278 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.1823 - accuracy: 0.9331 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1654 - accuracy: 0.9390 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1559 - accuracy: 0.9422 - 32s/epoch - 29ms/step
accuracy: 94.13%
INFO:tensorflow:Assets written to: ram://8d8c4694-cf7c-4d66-b995-f869968de8cd/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 33s - loss: 0.6800 - accuracy: 0.5677 - 33s/epoch - 29ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.5897 - accuracy: 0.6702 - 32s/epoch - 28ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.5207 - accuracy: 0.7349 - 32s/epoch - 28ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4028 - accuracy: 0.8327 - 32s/epoch - 28ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3159 - accuracy: 0.8768 - 32s/epoch - 28ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.2552 - accuracy: 0.9055 - 32s/epoch - 28ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2218 - accuracy: 0.9184 - 32s/epoch - 28ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2008 - accuracy: 0.9267 - 32s/epoch - 28ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1779 - accuracy: 0.9358 - 32s/epoch - 28ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1655 - accuracy: 0.9400 - 32s/epoch - 28ms/step
accuracy: 94.47%
INFO:tensorflow:Assets written to: ram://affe29a6-34c9-45f4-9431-80d111620818/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 33s - loss: 0.7134 - accuracy: 0.5340 - 33s/epoch - 29ms/step
Epoch 2/10
1122/1122 - 31s - loss: 0.6349 - accuracy: 0.6362 - 31s/epoch - 28ms/step
Epoch 3/10
1122/1122 - 31s - loss: 0.5546 - accuracy: 0.7186 - 31s/epoch - 28ms/step
Epoch 4/10
1122/1122 - 31s - loss: 0.4369 - accuracy: 0.8173 - 31s/epoch - 28ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.2868 - accuracy: 0.8963 - 32s/epoch - 28ms/step
Epoch 6/10
1122/1122 - 31s - loss: 0.2273 - accuracy: 0.9154 - 31s/epoch - 28ms/step
Epoch 7/10
1122/1122 - 31s - loss: 0.1976 - accuracy: 0.9276 - 31s/epoch - 28ms/step
Epoch 8/10
1122/1122 - 31s - loss: 0.1774 - accuracy: 0.9357 - 31s/epoch - 28ms/step
Epoch 9/10
1122/1122 - 31s - loss: 0.1640 - accuracy: 0.9399 - 31s/epoch - 28ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1492 - accuracy: 0.9455 - 32s/epoch - 28ms/step
accuracy: 94.96%
INFO:tensorflow:Assets written to: ram://c61f3f0a-e970-4914-9e8e-3779df9ed26f/assets
Model Accuracy List:  [81.51991963386536, 93.9838111400604

In [23]:
#RNN AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 28s - loss: 0.2888 - accuracy: 0.8637 - 28s/epoch - 25ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.0664 - accuracy: 0.9747 - 32s/epoch - 28ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.0367 - accuracy: 0.9860 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.0315 - accuracy: 0.9871 - 32s/epoch - 28ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.0259 - accuracy: 0.9892 - 32s/epoch - 28ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.0278 - accuracy: 0.9890 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.0213 - accuracy: 0.9910 - 32s/epoch - 28ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.0218 - accuracy: 0.9911 - 32s/epoch - 28ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.0179 - accuracy: 0.9925 - 32s/epoch - 28ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.0200 - accuracy: 0.9916 - 32s/epoch - 28ms/step
accuracy: 99.56%
INFO:tensorflow:Assets written to: ram://a0451170-f152-4ad3-93a8-d9e8eabe7633/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.1723 - accuracy: 0.9218 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.0506 - accuracy: 0.9810 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0352 - accuracy: 0.9862 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0293 - accuracy: 0.9882 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0268 - accuracy: 0.9891 - 33s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0243 - accuracy: 0.9901 - 33s/epoch - 30ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0221 - accuracy: 0.9906 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.0243 - accuracy: 0.9912 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.0168 - accuracy: 0.9930 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.0175 - accuracy: 0.9933 - 32s/epoch - 29ms/step
accuracy: 98.37%
INFO:tensorflow:Assets written to: ram://9157b692-8d81-4869-a30f-8b310dbdc945/assets
Epoch 1/10


...

1122/1122 - 34s - loss: 0.2086 - accuracy: 0.9040 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.0553 - accuracy: 0.9799 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.0408 - accuracy: 0.9848 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.0324 - accuracy: 0.9870 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0256 - accuracy: 0.9896 - 33s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.0249 - accuracy: 0.9901 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.0218 - accuracy: 0.9913 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.0222 - accuracy: 0.9910 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.0185 - accuracy: 0.9924 - 32s/epoch - 28ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.0209 - accuracy: 0.9915 - 32s/epoch - 29ms/step
accuracy: 99.23%
INFO:tensorflow:Assets written to: ram://20edccc0-7b0b-4cfb-bbc0-77925e3e9d47/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 35s - loss: 0.1722 - accuracy: 0.9212 - 35s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0486 - accuracy: 0.9813 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0353 - accuracy: 0.9863 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0280 - accuracy: 0.9887 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.0232 - accuracy: 0.9905 - 34s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0246 - accuracy: 0.9898 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0274 - accuracy: 0.9899 - 33s/epoch - 30ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0297 - accuracy: 0.9882 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0195 - accuracy: 0.9920 - 33s/epoch - 30ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0198 - accuracy: 0.9921 - 33s/epoch - 29ms/step
accuracy: 99.26%
INFO:tensorflow:Assets written to: ram://62376a7a-84aa-453b-b2e5-aebc9a24d7a5/assets
Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 33s - loss: 0.1827 - accuracy: 0.9196 - 33s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.0620 - accuracy: 0.9772 - 32s/epoch - 28ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.0389 - accuracy: 0.9853 - 32s/epoch - 28ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.0337 - accuracy: 0.9868 - 32s/epoch - 28ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.0272 - accuracy: 0.9892 - 32s/epoch - 28ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.0286 - accuracy: 0.9890 - 32s/epoch - 28ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.0245 - accuracy: 0.9902 - 32s/epoch - 28ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.0228 - accuracy: 0.9908 - 32s/epoch - 28ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.0228 - accuracy: 0.9908 - 32s/epoch - 28ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.0224 - accuracy: 0.9915 - 32s/epoch - 28ms/step
accuracy: 98.91%
INFO:tensorflow:Assets written to: ram://fde26bea-6c86-4b14-82b0-3c37f3f88f57/assets
Model Accuracy List:  [99.56135749816895, 98.3738124370575

In [24]:
#GRU BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 80s - loss: 0.6672 - accuracy: 0.5859 - 80s/epoch - 72ms/step
Epoch 2/10
1122/1122 - 80s - loss: 0.6581 - accuracy: 0.6080 - 80s/epoch - 71ms/step
Epoch 3/10
1122/1122 - 80s - loss: 0.6524 - accuracy: 0.6195 - 80s/epoch - 72ms/step
Epoch 4/10
1122/1122 - 80s - loss: 0.6473 - accuracy: 0.6284 - 80s/epoch - 71ms/step
Epoch 5/10
1122/1122 - 80s - loss: 0.6425 - accuracy: 0.6331 - 80s/epoch - 71ms/step
Epoch 6/10
1122/1122 - 82s - loss: 0.6380 - accuracy: 0.6376 - 82s/epoch - 73ms/step
Epoch 7/10
1122/1122 - 80s - loss: 0.6336 - accuracy: 0.6410 - 80s/epoch - 72ms/step
Epoch 8/10
1122/1122 - 80s - loss: 0.6294 - accuracy: 0.6432 - 80s/epoch - 72ms/step
Epoch 9/10
1122/1122 - 80s - loss: 0.6251 - accuracy: 0.6468 - 80s/epoch - 71ms/step
Epoch 10/10
1122/1122 - 80s - loss: 0.6209 - accuracy: 0.6512 - 80s/epoch - 71ms/step
accuracy: 65.86%


INFO:tensorflow:Assets written to: ram://4dd9e1eb-3959-4160-8ba3-e667121a86de/assets


INFO:tensorflow:Assets written to: ram://4dd9e1eb-3959-4160-8ba3-e667121a86de/assets


...

INFO:tensorflow:Assets written to: ram://e9988037-6f1d-4f51-9579-9c2d59cd330c/assets


INFO:tensorflow:Assets written to: ram://e9988037-6f1d-4f51-9579-9c2d59cd330c/assets


Model Accuracy List:  [65.86070656776428, 64.0775978565216, 62.07560896873474, 62.0256781578064, 64.81098532676697]
63.77% (+/- 1.51%)


INFO:tensorflow:Assets written to: ram://1bb0f9b1-5e6d-49ff-b1db-0ee05f2843c5/assets


INFO:tensorflow:Assets written to: ram://1bb0f9b1-5e6d-49ff-b1db-0ee05f2843c5/assets


In [25]:
#GRU AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 80s - loss: 0.1986 - accuracy: 0.9060 - 80s/epoch - 71ms/step
Epoch 2/10
1122/1122 - 81s - loss: 0.0573 - accuracy: 0.9776 - 81s/epoch - 72ms/step
Epoch 3/10
1122/1122 - 82s - loss: 0.0336 - accuracy: 0.9866 - 82s/epoch - 73ms/step
Epoch 4/10
1122/1122 - 82s - loss: 0.0248 - accuracy: 0.9902 - 82s/epoch - 73ms/step
Epoch 5/10
1122/1122 - 81s - loss: 0.0203 - accuracy: 0.9917 - 81s/epoch - 72ms/step
Epoch 6/10
1122/1122 - 81s - loss: 0.0160 - accuracy: 0.9937 - 81s/epoch - 72ms/step
Epoch 7/10
1122/1122 - 82s - loss: 0.0166 - accuracy: 0.9933 - 82s/epoch - 73ms/step
Epoch 8/10
1122/1122 - 81s - loss: 0.0149 - accuracy: 0.9943 - 81s/epoch - 72ms/step
Epoch 9/10
1122/1122 - 82s - loss: 0.0130 - accuracy: 0.9953 - 82s/epoch - 73ms/step
Epoch 10/10
1122/1122 - 82s - loss: 0.0146 - accuracy: 0.9940 - 82s/epoch - 73ms/step
accuracy: 99.89%


INFO:tensorflow:Assets written to: ram://8ba7b1d7-f990-4bf3-baf9-4d3fa9937c32/assets


INFO:tensorflow:Assets written to: ram://8ba7b1d7-f990-4bf3-baf9-4d3fa9937c32/assets


...

INFO:tensorflow:Assets written to: ram://de41a718-4f5c-478a-9136-75d03d6cc13b/assets


INFO:tensorflow:Assets written to: ram://de41a718-4f5c-478a-9136-75d03d6cc13b/assets


Model Accuracy List:  [99.88588094711304, 99.58631992340088, 99.70042705535889, 99.55777525901794, 99.70042705535889]
99.69% (+/- 0.12%)


INFO:tensorflow:Assets written to: ram://d54d8647-276e-434c-9806-a10b2d49bad5/assets


INFO:tensorflow:Assets written to: ram://d54d8647-276e-434c-9806-a10b2d49bad5/assets


In [26]:
#LSTM BEFORE TUNING 

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 94s - loss: 0.6805 - accuracy: 0.5113 - 94s/epoch - 84ms/step
Epoch 2/10
1122/1122 - 93s - loss: 0.6658 - accuracy: 0.5906 - 93s/epoch - 83ms/step
Epoch 3/10
1122/1122 - 92s - loss: 0.6520 - accuracy: 0.6767 - 92s/epoch - 82ms/step
Epoch 4/10
1122/1122 - 92s - loss: 0.6417 - accuracy: 0.6908 - 92s/epoch - 82ms/step
Epoch 5/10
1122/1122 - 92s - loss: 0.6312 - accuracy: 0.6962 - 92s/epoch - 82ms/step
Epoch 6/10
1122/1122 - 92s - loss: 0.6202 - accuracy: 0.6978 - 92s/epoch - 82ms/step
Epoch 7/10
1122/1122 - 94s - loss: 0.6087 - accuracy: 0.6987 - 94s/epoch - 84ms/step
Epoch 8/10
1122/1122 - 92s - loss: 0.5978 - accuracy: 0.6988 - 92s/epoch - 82ms/step
Epoch 9/10
1122/1122 - 92s - loss: 0.5881 - accuracy: 0.6982 - 92s/epoch - 82ms/step
Epoch 10/10
1122/1122 - 91s - loss: 0.5800 - accuracy: 0.6975 - 91s/epoch - 81ms/step
accuracy: 70.09%


INFO:tensorflow:Assets written to: ram://ad324467-23b9-4795-ba6b-a92359e73c77/assets


INFO:tensorflow:Assets written to: ram://ad324467-23b9-4795-ba6b-a92359e73c77/assets


...

INFO:tensorflow:Assets written to: ram://04c785f8-568c-4230-a920-96ba823e1cb8/assets


INFO:tensorflow:Assets written to: ram://04c785f8-568c-4230-a920-96ba823e1cb8/assets


Model Accuracy List:  [70.09379267692566, 70.32203078269958, 70.78102827072144, 69.43652033805847, 70.68117260932922]
70.26% (+/- 0.48%)


INFO:tensorflow:Assets written to: ram://16c10a56-b760-45e8-877b-e0caf5e56a44/assets


INFO:tensorflow:Assets written to: ram://16c10a56-b760-45e8-877b-e0caf5e56a44/assets


In [27]:
#LSTM AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


C:\Users\asus\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 123s - loss: 0.2293 - accuracy: 0.8985 - 123s/epoch - 110ms/step
Epoch 2/10
1122/1122 - 123s - loss: 0.0793 - accuracy: 0.9681 - 123s/epoch - 110ms/step
Epoch 3/10
1122/1122 - 122s - loss: 0.0467 - accuracy: 0.9818 - 122s/epoch - 109ms/step
Epoch 4/10
1122/1122 - 124s - loss: 0.0324 - accuracy: 0.9876 - 124s/epoch - 110ms/step
Epoch 5/10
1122/1122 - 124s - loss: 0.0282 - accuracy: 0.9887 - 124s/epoch - 110ms/step
Epoch 6/10
1122/1122 - 124s - loss: 0.0255 - accuracy: 0.9898 - 124s/epoch - 110ms/step
Epoch 7/10
1122/1122 - 123s - loss: 0.0220 - accuracy: 0.9913 - 123s/epoch - 109ms/step
Epoch 8/10
1122/1122 - 123s - loss: 0.0200 - accuracy: 0.9920 - 123s/epoch - 110ms/step
Epoch 9/10
1122/1122 - 127s - loss: 0.0174 - accuracy: 0.9932 - 127s/epoch - 113ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.0159 - accuracy: 0.9933 - 125s/epoch - 111ms/step
accuracy: 99.56%


INFO:tensorflow:Assets written to: ram://1adbd81a-e45a-4ff3-b953-8ccf0ce5c0dd/assets


INFO:tensorflow:Assets written to: ram://1adbd81a-e45a-4ff3-b953-8ccf0ce5c0dd/assets


...

INFO:tensorflow:Assets written to: ram://3da61389-4861-4fb6-a4cc-2153627f95bb/assets


INFO:tensorflow:Assets written to: ram://3da61389-4861-4fb6-a4cc-2153627f95bb/assets


Model Accuracy List:  [99.56492185592651, 99.68617558479309, 99.5078444480896, 99.45791959762573, 99.22610521316528]
99.49% (+/- 0.15%)


INFO:tensorflow:Assets written to: ram://5625665a-27d2-41fe-b896-b63fd94bcc3f/assets


INFO:tensorflow:Assets written to: ram://5625665a-27d2-41fe-b896-b63fd94bcc3f/assets


Get list of accuracy, precision, recall and f1
Pengujian Data Testing
- Load Model untuk klasifikasi data Testing
- Model sudah di Load lalu melakukan Predict terhadap data Testing
- Simpan data Hasil Predict untuk codingan selanjutnya

In [40]:
#RNN K-FOLD BEFORE TUNING 
file_rnn_fold_before = [("/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_rnn_before = [(pickle.load(open(file_rnn_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_before = [(loaded_model_rnn_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_before = [(np.argmax(y_pred_rnn_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_before[ypvc] = np.reshape(y_pred_values_rnn_before[ypvc],(-1,1))
result_rnn_before = [(classification_report(y_test, y_pred_values_rnn_before[res], output_dict = True)) for res in range(total_fold)]

#RNN K-FOLD AFTER TUNING 
file_rnn_fold_after = [("/Users/asus/notebook/TA/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_rnn_after = [(pickle.load(open(file_rnn_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_after = [(loaded_model_rnn_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_after = [(np.argmax(y_pred_rnn_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_after[ypvc] = np.reshape(y_pred_values_rnn_after[ypvc],(-1,1))
result_rnn_after = [(classification_report(y_test, y_pred_values_rnn_after[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD BEFORE TUNING
file_gru_fold_before = [("/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_gru_before = [(pickle.load(open(file_gru_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_before = [(loaded_model_gru_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_before = [(np.argmax(y_pred_gru_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_before[ypvc] = np.reshape(y_pred_values_gru_before[ypvc],(-1,1))
result_gru_before = [(classification_report(y_test, y_pred_values_gru_before[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD AFTER TUNING
file_gru_fold_after = [("/Users/asus/notebook/TA/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_gru_after = [(pickle.load(open(file_gru_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_after = [(loaded_model_gru_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_after = [(np.argmax(y_pred_gru_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_after[ypvc] = np.reshape(y_pred_values_gru_after[ypvc],(-1,1))
result_gru_after = [(classification_report(y_test, y_pred_values_gru_after[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD BEFORE TUNING
file_lstm_fold_before = [("/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_lstm_before = [(pickle.load(open(file_lstm_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_before = [(loaded_model_lstm_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_before = [(np.argmax(y_pred_lstm_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_before[ypvc] = np.reshape(y_pred_values_lstm_before[ypvc],(-1,1))
result_lstm_before = [(classification_report(y_test, y_pred_values_lstm_before[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD AFTER TUNING
file_lstm_fold_after = [("/Users/asus/notebook/TA/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fn+1) + ".h5") for fn in range(total_fold)]
loaded_model_lstm_after = [(pickle.load(open(file_lstm_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_after = [(loaded_model_lstm_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_after = [(np.argmax(y_pred_lstm_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_after[ypvc] = np.reshape(y_pred_values_lstm_after[ypvc],(-1,1))
result_lstm_after = [(classification_report(y_test, y_pred_values_lstm_after[res], output_dict = True)) for res in range(total_fold)]

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://3c582e07-dee4-483d-bd9f-eb1a7db92c10/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

Buat List untuk menampung dari Hasil dari Fold untuk Precision,Recall,Accuracy

In [ ]:
precision_list_before = {}
precision_list_after = {}
recall_list_before = {}
recall_list_after = {}
accuracy_list_before = {}
accuracy_list_after = {}
f1_list_before = {}
f1_list_after = {}

Rata-rata dari Precision,Recall,F1-Score,Accuracy

Mengambil data hasil predict pada codingan sebelumnya


In [ ]:
for sco in range(total_fold):
  precision_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['precision'] , result_gru_before[sco]['weighted avg']['precision'], result_lstm_before[sco]['weighted avg']['precision']]
  precision_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['precision'] , result_gru_after[sco]['weighted avg']['precision'], result_lstm_after[sco]['weighted avg']['precision']]
  recall_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['recall'] , result_gru_before[sco]['weighted avg']['recall'], result_lstm_before[sco]['weighted avg']['recall']]
  recall_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['recall'] , result_gru_after[sco]['weighted avg']['recall'], result_lstm_after[sco]['weighted avg']['recall']]
  f1_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['f1-score'] , result_gru_before[sco]['weighted avg']['f1-score'], result_lstm_before[sco]['weighted avg']['f1-score']]
  f1_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['f1-score'] , result_gru_after[sco]['weighted avg']['f1-score'], result_lstm_after[sco]['weighted avg']['f1-score']]
  accuracy_list_before[(sco+1)] = [result_rnn_before[sco]['accuracy'] , result_gru_before[sco]['accuracy'], result_lstm_before[sco]['accuracy']]
  accuracy_list_after[(sco+1)] = [result_rnn_after[sco]['accuracy'] , result_gru_after[sco]['accuracy'], result_lstm_after[sco]['accuracy']]

Menampilkan Hasil Predict Tiap Fold

In [ ]:
print("{:<5} {:<0}".format("","Precision K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Recall K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Accuracy K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","F1-Score K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")